<h1 align = "center",style ="font-size":40px>Merton Model for Credit Risk</h1>


<h2 style = "font-size:25px"><ins>Theory</ins></h2> 

The Black-Scholes formula is an expression for the price of a European call option given the price $S$ and volatility($\sigma$) of the underlying asset, as well as the maturiy time $T$:

<a id="eq-BSF"></a>
$$
\begin{equation}\tag{1}
C(0,S) = S_0 N(d_1) - K e^{-rT} N(d_2)
\end{equation}
$$
where 

$$
\begin{align*}
d_1 &= \frac{\log(S_0/K) + (r+ \sigma^2/2)T}{\sigma \sqrt{T}}\\
d_2 &= d_1 - \sigma \sqrt{T}
\end{align*}
$$
It is derived by assuming that the stock price is modelled by the stochastic differential equation $dS_t = \mu S_t dt + \sigma S_t dW_t$, which has the solution

<a id="eq-S_T"></a>
$$
\begin{equation}\tag{2}
\ln(S_T) = \ln(S_0) + (r - \frac{1}{2}\sigma^2)T + \sigma \sqrt{T} Z
\end{equation}
$$

where $Z \sim \mathcal{N}(0,1)$ is a normal random variable. Under the risk-neutral measure($\mu \to r$), we arrive at a simple interpretation for $N(d_2)$ where $N(x)$ is the standard cumulative distribution function. That is, it's the probability that a standard random variable $\mathcal{N}$ has a value smaller than $x$:
$$
N(x) = \mathbb{P}(Z<x)
$$
Since $\mathcal{N}$ is symmetric about $0$,
$$
\begin{align*}
N(d_2) &= \mathbb{P}(Z< d_2)\\
&= \mathbb{P}(Z > -d_2)\\
&= \mathbb{P}\left(Z > -\frac{\log(S_0/K) - (r - \sigma^2/2)T}{\sigma \sqrt{T}} \right)\\
\end{align*}
$$

Substituting the expression for $Z$ from <a href="#eq-S_T"> (2) </a>,
$$
\begin{align*}
N(d_2) &= \mathbb{P}(\ln(S_T)> \ln(K))\\
       &= \mathbb{P}(S_T> K)
\end{align*}
$$
Therefore, $N(d_2)$ is the probability that the option is in the money at maturity. Merton<a href="#ref1"><sup>[1]</sup></a> proposed that default probabilities could be estimated by modelling the equity of a firm as an option on its assets, with the firm's debt as the strike price. He assumed that the firm can neither pay out dividends nor issue new debt until the maturity of its current debt $D$, and that the market is frictionless(that is, the total asset value equals the sum of total equity and total debt). Consequently, there are two possibilities when the debt matures(at time $T$):
* The value of assets is larger than the debt, in which case the debt $D$ is paid back to lenders, and shareholders are left with $A_T - D$ in equity
* The debt exceeds the value of assets, leaving lenders with the firm's assets $A_T$ to liquidate, and the shareholders receive nothing

Mathematically, $E_T = \text{max}\{ (A_T - D), 0 \}$ and $D = \text{min}\{ A_T,D\}$.

Applying the Black-Scholes call formula, the initial(current) value of equity is 
<a id="eq-EV"></a>
$$
\begin{equation}\tag{3}
E_0 = V_0 N(d_1) - D e^{-rT}N(d_2)
\end{equation}
$$
where 
$$
\begin{align*}
d_1 &= \frac{\log(V_0/D) + (r+ \sigma_V^2/2)(T-t)}{\sigma_V \sqrt{T}}\\
d_2 &= d_1 - \sigma_V \sqrt{T}
\end{align*}
$$
The default probability is then given by $\mathbb{P}(V_T < D)$, which is simply the probability that the option is out of the money:
$$
\begin{align*}
PD &= 1 - \mathbb{P}(V_T > D) \\
   &= N(-d_2)
\end{align*}
$$
as discussed above. To calculate this probability, we need $V_0$ and $\sigma_V$, neither of which are observable. In a risk-neutral scenario like ours, where we've hedged the option price against the underlying asset(that's how  <a href="#eq-BSF"> (1) </a> is derived: see <a href="https://github.com/praveer-kg/Finance_Public/blob/main/Option%20Pricing/1.%20Black-Scholes%20and%20Implied%20Volatility.ipynb">this notebook</a>), we can apply Itô's lemma to obtain the following relationship between the call price and asset price:


$$
\sigma_E E = \frac{\partial E}{\partial V} \sigma_V V
$$
At the initial time,
<a id="eq-Sec"></a>
$$
\begin{equation}\tag{4}
\sigma_E E_0 = N(d_1) \sigma_V V_0
\end{equation}
$$
which gives us a second equation (following <a href="#eq-EV"> (3) </a>) to solve for the two unknown variables $\sigma_V$ and $V_0$. These values can then be used to calculate $N(-d_2)$, which is the probability of default.

<h2 style = "font-size:25px"><ins>Code</ins></h2> 

I will implement Merton's model for a few stocks using Python. First, let's import some modules to solve <a href="#eq-EV"> (3) </a> and <a href="#eq-Sec"> (4) </a>, calculate $N(.)$ and query stock data to obtain $\sigma_E$ from. 

In [1]:
from scipy.optimize import root
from scipy.stats import norm
import numpy as np
import yfinance as yf
import pandas as pd

The function below implements a root solving algorithm to solve the transcendental equations <a href="#eq-EV"> (3) </a> and <a href="#eq-Sec"> (4) </a>. It reads in the relevant constants and returns a pair of equations in $V_0$ and $\sigma_V$, which are then supplied to ```scipy.optimize.root```, along with the initial guesses for $V_0$ and $\sigma_V$.

In [2]:
def Merton_Solve(vars, E0, σ_E, D, r, T):
    V0, σ_V = vars
    d1 = (np.log(V0 / D) + (r + 0.5 * σ_V**2) * T) / (σ_V * np.sqrt(T))
    d2 = (np.log(V0 / D) + (r - 0.5 * σ_V**2) * T) / (σ_V * np.sqrt(T))

    f1 = V0 * norm.cdf(d1) - D * np.exp(-r * T) * norm.cdf(d2) - E0
    f2 = σ_V * V0 * norm.cdf(d1) - σ_E * E0
    return [f1, f2]

I will first apply the model to Apple stocks using $\texttt{yfinance}$. The debt $D$ is listed as $\$176$bn on their financial statement dated 28 September 2024<a href="#ref2"><sup>[2]</sup></a>. The market cap $E_0$ was $\$3500$bn on that date according to public records. Finally, the volatility $\sigma_E$ can be calculated using stocks data provided by $\texttt{yfinance}$, and the riks-free rate is set to $4.2\%$.

In [3]:
#calculate volatility
ticker = "AAPL"
start =pd.Timestamp(2024,9,1)
stop = pd.Timestamp("today")
data = yf.download(ticker, start, stop)    #download data
close_prices = data["Close"]
returns = close_prices.pct_change().dropna()
returns = returns[ticker].to_numpy()

σ_E = np.std(returns)*np.sqrt(252)
E0 = 3500
D = 176
r = 0.042
T = 1
initial_guess = [1.5 * E0, σ_E]  
sol = root(Merton_Solve, initial_guess, args=(E0, σ_E, D, r, T))
V0, σ_V = sol.x

P = norm.cdf(-(np.log(V0 / D) + (r - 0.5 * σ_V**2) * T) / (σ_V * np.sqrt(T)))
print("Probability of Default =",P)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Probability of Default = 2.2887093283883895e-19


The probability is close to zero(~$ 10^{-19}$ on my local machine) as expected, given that Apple is "too big to fail". 

Now let's analyze Carnival Corporation instead, a company which was labelled a "corporate zombie" by Bloomberg<a href="#ref4"><sup>[3]</sup></a> . Its financial statement<a href="#ref4"><sup>[4]</sup></a> lists current liabilities totalling $11.6$bn as of November 2024, at which point its market cap was roughly $\$34$bn.

In [4]:
#calculate volatility
ticker = "CCL"
start =pd.Timestamp(2024,10,1)
stop = pd.Timestamp("today")
data = yf.download(ticker, start, stop)    #download data
close_prices = data["Close"]
returns = close_prices.pct_change().dropna()
returns = returns[ticker].to_numpy()

σ_E = np.std(returns)*np.sqrt(252)
E0 = 35
D = 11.6
r = 0.042
T = 1
initial_guess = [1.5 * E0, σ_E]  
sol = root(Merton_Solve, initial_guess, args=(E0, σ_E, D, r, T))
V0, σ_V = sol.x

P = norm.cdf(-(np.log(V0 / D) + (r - 0.5 * σ_V**2) * T) / (σ_V * np.sqrt(T)))
print("Probability of Default =",P)

[*********************100%***********************]  1 of 1 completed

Probability of Default = 0.00045003553641007776


The probability of default is about $15$ orders of magnitude higher, in line with perceptions of the company.

## References
<a id="ref1"></a>[1]  Merton, R.C. (1974), "<a href="https://onlinelibrary.wiley.com/doi/10.1111/j.1540-6261.1974.tb03058.x">*ON THE PRICING OF CORPORATE DEBT: THE RISK STRUCTURE OF INTEREST RATES*</a>" The Journal of Finance, 29: 449-470.

<a id="ref2"></a>[2]  "<a href="https://www.apple.com/newsroom/2024/10/apple-reports-fourth-quarter-results/">*Apple reports fourth quarter results*</a>", apple.com

<a id="ref3"></a>[3]  "<a href="https://www.bloomberg.com/news/articles/2020-11-17/america-s-zombie-companies-have-racked-up-1-4-trillion-of-debt">*America’s Zombie Companies Rack Up $2 Trillion of Debt*</a>", bloomberg.com

<a id="ref4"></a>[4]  "<a href="https://www.carnivalcorp.com/wp-content/uploads/2024/08/Carnival-Corporation-plc-2024-Annual-Report.pdf">*Carnival Corporation \& PLC 2024 ANNUAL REPORT*</a>", carnivalcorp.com